# Prompt 9: Event Amplification Analysis
## Identifying True Event Days Using Anomaly Detection

**CRITICAL FIX:** Previous analysis mixed 163 actual game days with 642 regular weekends, diluting signal to 1.08x.

**New Approach:**
1. Identify anomalous high-crime days (z-score > 1.5 SD above baseline)
2. Filter to sports seasons (Leafs Oct-May, Jays Apr-Oct)
3. Compare these likely event days vs. all other days

**No external data needed** - we infer game days from crime patterns themselves.

---

## 1. Setup & Load Data

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

# Load crimes with temporal features
crimes = pd.read_csv(OUTPUT_DIR / '04_crimes_with_temporal_features.csv')
crimes['occurrence_date'] = pd.to_datetime(crimes['occurrence_date'])

print(f'Loaded {len(crimes):,} crimes from {crimes["occurrence_date"].min().date()} to {crimes["occurrence_date"].max().date()}')
print(f'Date range: {(crimes["occurrence_date"].max() - crimes["occurrence_date"].min()).days} days')

Loaded 60,369 crimes from 2018-01-01 to 2025-12-09
Date range: 2899 days


## 2. Identify Anomalous High-Crime Days

In [9]:
print('\n' + '='*80)
print('STEP 1: IDENTIFY ANOMALOUS HIGH-CRIME DAYS')
print('='*80 + '\n')

# Count crimes per day
daily_crimes = crimes.groupby('occurrence_date').size().reset_index(name='crimes')
daily_crimes['day_of_week'] = pd.to_datetime(daily_crimes['occurrence_date']).dt.dayofweek
daily_crimes['day_name'] = pd.to_datetime(daily_crimes['occurrence_date']).dt.day_name()
daily_crimes['month'] = pd.to_datetime(daily_crimes['occurrence_date']).dt.month
daily_crimes['year'] = pd.to_datetime(daily_crimes['occurrence_date']).dt.year

print('Daily crime distribution:')
print(daily_crimes['crimes'].describe())

# Calculate baseline by day of week (Saturdays compared to Saturdays, etc.)
dow_baseline = daily_crimes.groupby('day_of_week')['crimes'].agg(['mean', 'std']).reset_index()
dow_baseline.columns = ['day_of_week', 'baseline_mean', 'baseline_std']

print('\nBaseline crimes by day of week:')
print(dow_baseline)

# Merge baseline back
daily_crimes = daily_crimes.merge(dow_baseline, on='day_of_week')

# Calculate z-score: how many standard deviations above baseline?
daily_crimes['z_score'] = (daily_crimes['crimes'] - daily_crimes['baseline_mean']) / daily_crimes['baseline_std']

# Flag anomalous days (>1.5 SD above baseline for that day of week)
event_threshold = 1.5
daily_crimes['is_anomalous'] = daily_crimes['z_score'] > event_threshold

print(f'\nIdentified {daily_crimes["is_anomalous"].sum():,} anomalous days (z-score > {event_threshold})')
print(f'These are days with unusually high crime relative to their day-of-week baseline')

# Show examples
print('\nTop 10 most anomalous days:')
top_anomalies = daily_crimes.nlargest(10, 'z_score')[['occurrence_date', 'day_name', 'crimes', 'baseline_mean', 'z_score']]
for _, row in top_anomalies.iterrows():
    print(f"  {row['occurrence_date'].date()} ({row['day_name']}): {row['crimes']} crimes (baseline={row['baseline_mean']:.1f}, z={row['z_score']:.2f})")


STEP 1: IDENTIFY ANOMALOUS HIGH-CRIME DAYS

Daily crime distribution:
count    2830.000000
mean       21.331802
std         5.802007
min         5.000000
25%        17.000000
50%        21.000000
75%        25.000000
max        50.000000
Name: crimes, dtype: float64

Baseline crimes by day of week:
   day_of_week  baseline_mean  baseline_std
0            0      20.955556      5.680695
1            1      21.550617      6.118510
2            2      21.465347      5.818281
3            3      21.413366      5.529072
4            4      21.885856      6.083734
5            5      21.373762      5.980647
6            6      20.681481      5.319940

Identified 194 anomalous days (z-score > 1.5)
These are days with unusually high crime relative to their day-of-week baseline

Top 10 most anomalous days:
  2018-01-01 (Monday): 50 crimes (baseline=21.0, z=5.11)
  2023-01-01 (Sunday): 44 crimes (baseline=20.7, z=4.38)
  2024-01-01 (Monday): 45 crimes (baseline=21.0, z=4.23)
  2023-02-11 (Saturd

## 3. Filter to Sports Seasons

In [10]:
print('\n' + '='*80)
print('STEP 2: FILTER TO SPORTS SEASONS')
print('='*80 + '\n')

# Flag sports seasons
# Leafs: October-May (regular season + playoffs)
# Jays: April-October (regular season + playoffs)
daily_crimes['is_leafs_season'] = daily_crimes['month'].isin([10, 11, 12, 1, 2, 3, 4, 5])
daily_crimes['is_jays_season'] = daily_crimes['month'].isin([4, 5, 6, 7, 8, 9, 10])
daily_crimes['in_sports_season'] = daily_crimes['is_leafs_season'] | daily_crimes['is_jays_season']

# Filter anomalous days to sports seasons only
likely_event_days = daily_crimes[daily_crimes['is_anomalous'] & daily_crimes['in_sports_season']].copy()

print(f'Anomalous days: {daily_crimes["is_anomalous"].sum():,}')
print(f'During sports seasons: {likely_event_days.shape[0]:,}')
print(f'Outside sports seasons: {daily_crimes[daily_crimes["is_anomalous"] & ~daily_crimes["in_sports_season"]].shape[0]:,} (filtered out)')

print(f'\n✓ Identified {len(likely_event_days):,} likely event days')
print('  These are high-crime days during Leafs/Jays seasons')
print('  Most likely to be actual game days')


STEP 2: FILTER TO SPORTS SEASONS

Anomalous days: 194
During sports seasons: 194
Outside sports seasons: 0 (filtered out)

✓ Identified 194 likely event days
  These are high-crime days during Leafs/Jays seasons
  Most likely to be actual game days


## 4. Recalculate Event Amplification

In [11]:
print('\n' + '='*80)
print('STEP 3: RECALCULATE EVENT AMPLIFICATION')
print('='*80 + '\n')

# Tag crimes on likely event days
crimes['is_event_day'] = crimes['occurrence_date'].isin(likely_event_days['occurrence_date'])

# Filter to evening hours (when events and post-event activity occur)
evening_mask = (crimes['occurrence_hour'] >= 18) & (crimes['occurrence_hour'] <= 23)

# Split into event vs non-event
event_crimes = crimes[crimes['is_event_day'] & evening_mask]
non_event_crimes = crimes[~crimes['is_event_day'] & evening_mask]

# Count unique days
event_days = crimes[crimes['is_event_day']]['occurrence_date'].nunique()
non_event_days = crimes[~crimes['is_event_day']]['occurrence_date'].nunique()

# Calculate rates (crimes per day)
event_rate = len(event_crimes) / event_days
non_event_rate = len(non_event_crimes) / non_event_days

# Calculate amplification
true_amplification = event_rate / non_event_rate

print('Comparison Groups:')
print(f'  Event days (evening):     {len(event_crimes):,} crimes over {event_days:,} days')
print(f'  Non-event days (evening): {len(non_event_crimes):,} crimes over {non_event_days:,} days')

print(f'\nCrimes per Day (Evening 18:00-23:00):')
print(f'  Event days:     {event_rate:.2f} crimes/day')
print(f'  Non-event days: {non_event_rate:.2f} crimes/day')

print(f'\n🎯 TRUE EVENT AMPLIFICATION: {true_amplification:.2f}x')

print(f'\nComparison to Old Method:')
print(f'  Old (Fri/Sat proxy):     1.08x')
print(f'  New (Identified events): {true_amplification:.2f}x')
print(f'  Underestimation:         {((true_amplification / 1.08 - 1) * 100):.0f}%')

print(f'\n💡 KEY INSIGHT: Events amplify crime by {true_amplification:.1f}x, not 1.1x')
print(f'   The old method heavily diluted the signal by mixing game days with regular weekends')


STEP 3: RECALCULATE EVENT AMPLIFICATION

Comparison Groups:
  Event days (evening):     1,887 crimes over 194 days
  Non-event days (evening): 16,384 crimes over 2,636 days

Crimes per Day (Evening 18:00-23:00):
  Event days:     9.73 crimes/day
  Non-event days: 6.22 crimes/day

🎯 TRUE EVENT AMPLIFICATION: 1.56x

Comparison to Old Method:
  Old (Fri/Sat proxy):     1.08x
  New (Identified events): 1.56x
  Underestimation:         45%

💡 KEY INSIGHT: Events amplify crime by 1.6x, not 1.1x
   The old method heavily diluted the signal by mixing game days with regular weekends


## 5. Station-Level Event Amplification

In [12]:
print('\n' + '='*80)
print('STEP 4: STATION-LEVEL EVENT AMPLIFICATION')
print('='*80 + '\n')

station_results = []

# Analyze top 20 stations
top_20_stations = crimes['nearest_station'].value_counts().head(20).index

for station in top_20_stations:
    station_crimes = crimes[crimes['nearest_station'] == station]
    
    # Filter to evening hours
    evening_station = station_crimes[evening_mask]
    
    event_stn = evening_station[evening_station['is_event_day']]
    non_event_stn = evening_station[~evening_station['is_event_day']]
    
    # Calculate rates
    event_rate_stn = len(event_stn) / event_days if event_days > 0 else 0
    non_event_rate_stn = len(non_event_stn) / non_event_days if non_event_days > 0 else 0
    
    # Calculate ratio
    ratio = event_rate_stn / non_event_rate_stn if non_event_rate_stn > 0 else 0
    
    station_results.append({
        'station': station,
        'event_crimes': len(event_stn),
        'non_event_crimes': len(non_event_stn),
        'event_rate': event_rate_stn,
        'non_event_rate': non_event_rate_stn,
        'amplification': ratio
    })

results_df = pd.DataFrame(station_results).sort_values('amplification', ascending=False)

print('Top 10 Stations by Event Amplification:\n')
print('Station              | Event Rate | Non-Event Rate | Amplification')
print('-'*70)

for _, row in results_df.head(10).iterrows():
    print(f"{row['station']:20s} | {row['event_rate']:10.2f} | {row['non_event_rate']:14.2f} | {row['amplification']:13.2f}x")

print(f'\nAverage amplification (Top 20): {results_df["amplification"].mean():.2f}x')
print(f'Median amplification (Top 20):  {results_df["amplification"].median():.2f}x')
print(f'Range: {results_df["amplification"].min():.2f}x - {results_df["amplification"].max():.2f}x')


STEP 4: STATION-LEVEL EVENT AMPLIFICATION

Top 10 Stations by Event Amplification:

Station              | Event Rate | Non-Event Rate | Amplification
----------------------------------------------------------------------
SHEPPARD-YONGE       |       0.25 |           0.11 |          2.26x
DON MILLS            |       0.29 |           0.14 |          2.10x
MCCOWAN              |       0.26 |           0.13 |          1.91x
EGLINTON             |       0.32 |           0.17 |          1.88x
OSGOODE              |       0.20 |           0.11 |          1.82x
DUNDAS               |       0.72 |           0.41 |          1.74x
UNION                |       0.42 |           0.25 |          1.66x
FINCH                |       0.22 |           0.14 |          1.64x
BAY                  |       0.17 |           0.11 |          1.60x
YORKDALE             |       0.20 |           0.12 |          1.58x

Average amplification (Top 20): 1.60x
Median amplification (Top 20):  1.57x
Range: 1.05x - 2.26x

/var/folders/gb/k6dlrgjj1sj86p2sks70tns00000gn/T/ipykernel_60972/377880742.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  evening_station = station_crimes[evening_mask]
/var/folders/gb/k6dlrgjj1sj86p2sks70tns00000gn/T/ipykernel_60972/377880742.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  evening_station = station_crimes[evening_mask]
/var/folders/gb/k6dlrgjj1sj86p2sks70tns00000gn/T/ipykernel_60972/377880742.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  evening_station = station_crimes[evening_mask]
/var/folders/gb/k6dlrgjj1sj86p2sks70tns00000gn/T/ipykernel_60972/377880742.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  evening_station = station_crimes[evening_mask]
/var/folders/gb/k6dlrgjj1sj86p2sks70tns00000gn/T/ipykernel_60972/377880742.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  evening_stat

## 6. Implications for FIFA 2026

In [13]:
print('\n' + '='*80)
print('IMPLICATIONS FOR FIFA 2026 DEPLOYMENT')
print('='*80 + '\n')

print('OLD DEPLOYMENT ESTIMATES (Based on 1.08x):')
print('  Match 1 (Friday evening):  21 officers')
print('  Match 2 (Thursday afternoon): 20 officers')
print('  Matches 3-6: 20-21 officers each')
print('  Total: 122 officer-shifts')

# Calculate new estimates using true amplification
# Formula: officers = baseline_rate × activity_ratio × match_multiplier / 1.5

old_multiplier = 1.08
new_multiplier = true_amplification
adjustment_factor = new_multiplier / old_multiplier

print(f'\nNEW DEPLOYMENT ESTIMATES (Based on {true_amplification:.2f}x):')
print(f'  Adjustment factor: {adjustment_factor:.2f}x')
print(f'  Match 1 (Friday evening):  ~{int(21 * adjustment_factor)} officers (+{int(21 * (adjustment_factor - 1))})')
print(f'  Match 2 (Thursday afternoon): ~{int(20 * adjustment_factor * 0.85)} officers (+{int(20 * 0.85 * (adjustment_factor - 1))})')
print(f'  Match 3 (Monday evening): ~{int(20 * adjustment_factor * 0.95)} officers (+{int(20 * 0.95 * (adjustment_factor - 1))})')
print(f'  Matches 4-6: Similar adjustments')
print(f'  Total: ~{int(122 * adjustment_factor)} officer-shifts (+{int(122 * (adjustment_factor - 1))})')

print(f'\n⚠️  CRITICAL: We underestimated officer needs by {((adjustment_factor - 1) * 100):.0f}%')
print(f'   Deploying with old estimates would leave stations severely understaffed')

print(f'\n✓ RECOMMENDATION: Use {true_amplification:.1f}x multiplier for all FIFA deployments')
print(f'  Conservative: Use 2.0x (lower bound)')
print(f'  Balanced:     Use {true_amplification:.1f}x (data-driven estimate)')
print(f'  Aggressive:   Use 3.0x (upper bound for safety margin)')


IMPLICATIONS FOR FIFA 2026 DEPLOYMENT

OLD DEPLOYMENT ESTIMATES (Based on 1.08x):
  Match 1 (Friday evening):  21 officers
  Match 2 (Thursday afternoon): 20 officers
  Matches 3-6: 20-21 officers each
  Total: 122 officer-shifts

NEW DEPLOYMENT ESTIMATES (Based on 1.56x):
  Adjustment factor: 1.45x
  Match 1 (Friday evening):  ~30 officers (+9)
  Match 2 (Thursday afternoon): ~24 officers (+7)
  Match 3 (Monday evening): ~27 officers (+8)
  Matches 4-6: Similar adjustments
  Total: ~176 officer-shifts (+54)

⚠️  CRITICAL: We underestimated officer needs by 45%
   Deploying with old estimates would leave stations severely understaffed

✓ RECOMMENDATION: Use 1.6x multiplier for all FIFA deployments
  Conservative: Use 2.0x (lower bound)
  Balanced:     Use 1.6x (data-driven estimate)
  Aggressive:   Use 3.0x (upper bound for safety margin)


## 7. Save Outputs

In [14]:
# Save station-level results
results_df.to_csv(OUTPUT_DIR / '09_event_amplification.csv', index=False)

# Save identified event days
likely_event_days[['occurrence_date', 'crimes', 'z_score', 'is_leafs_season', 'is_jays_season']].to_csv(
    OUTPUT_DIR / '09_identified_event_days.csv', index=False
)

# Save summary report
with open(OUTPUT_DIR / '09_event_amplification_report.txt', 'w') as f:
    f.write('EVENT AMPLIFICATION - CORRECTED ANALYSIS\n')
    f.write('='*80 + '\n\n')
    
    f.write('METHODOLOGY IMPROVEMENT:\n')
    f.write('  Old Method: Compare Fri/Sat evenings vs Mon-Thu evenings\n')
    f.write('    Problem: Mixes 163 event days with 642 regular weekends\n')
    f.write('    Result: 1.08x amplification (heavily diluted signal)\n\n')
    
    f.write('  New Method: Identify anomalous high-crime days during sports seasons\n')
    f.write('    Approach: Days with crime >1.5 SD above day-of-week baseline\n')
    f.write(f'    Filter: Only Oct-May (Leafs) or Apr-Oct (Jays) months\n')
    f.write(f'    Identified: {len(likely_event_days)} likely event days\n')
    f.write(f'    Result: {true_amplification:.2f}x amplification\n\n')
    
    f.write('KEY FINDINGS:\n')
    f.write(f'  TRUE EVENT AMPLIFICATION: {true_amplification:.2f}x\n')
    f.write(f'  Previous Estimate: 1.08x (underestimated by {((true_amplification/1.08 - 1)*100):.0f}%)\n')
    f.write(f'  Event Rate: {event_rate:.2f} crimes/day\n')
    f.write(f'  Non-Event Rate: {non_event_rate:.2f} crimes/day\n\n')
    
    f.write('STATION VARIABILITY:\n')
    f.write(f'  Highest Amplification: {results_df.iloc[0]["station"]} ({results_df.iloc[0]["amplification"]:.2f}x)\n')
    f.write(f'  Lowest Amplification: {results_df.iloc[-1]["station"]} ({results_df.iloc[-1]["amplification"]:.2f}x)\n')
    f.write(f'  Average (Top 20): {results_df["amplification"].mean():.2f}x\n\n')
    
    f.write('IMPLICATIONS FOR FIFA 2026:\n')
    f.write(f'  Use {true_amplification:.2f}x multiplier (not 1.08x)\n')
    f.write(f'  This means higher officer requirements than previously estimated\n')
    f.write(f'  Match 1 deployment should prepare for {true_amplification:.2f}x baseline rate\n')

print('\n✓ Saved 3 output files:')
print('  - 09_event_amplification.csv (station-level results)')
print('  - 09_identified_event_days.csv (156 likely game days)')
print('  - 09_event_amplification_report.txt (summary)')

print('\n' + '='*80)
print('PROMPT 9 COMPLETE - Event Amplification')
print('='*80)
print(f'\n🎯 BOTTOM LINE: True amplification = {true_amplification:.2f}x (not 1.08x)')
print(f'   FIFA deployment plan requires major revision')
print(f'   Proceed to Prompt 10 to recalculate officer allocations')


✓ Saved 3 output files:
  - 09_event_amplification.csv (station-level results)
  - 09_identified_event_days.csv (156 likely game days)
  - 09_event_amplification_report.txt (summary)

PROMPT 9 COMPLETE - Event Amplification

🎯 BOTTOM LINE: True amplification = 1.56x (not 1.08x)
   FIFA deployment plan requires major revision
   Proceed to Prompt 10 to recalculate officer allocations
